In [1]:
import numpy as np
import xarray as xr
import time 
import os
from netCDF4 import Dataset
from pyproj import CRS
import rasterio
import rioxarray
from rasterio.enums import Resampling
from sklearn.preprocessing import StandardScaler


In [4]:
#For calculating statistical northness and eastness

filename_physstates_2d = '/cluster/projects/itk-SINMOD/coral-mapping/midnor/samp_2D_jan_jun.nc'
physstates_2d = Dataset(filename_physstates_2d, 'r')
gridLons = physstates_2d.variables['gridLons']

In [2]:
filename_sinmod = '/cluster/projects/itk-SINMOD/coral-mapping/midnor/PhysStates_2019.nc'
sinmod_2019 = xr.open_dataset(filename_sinmod)

In [3]:
print(list(sinmod_2019.variables))

['time', 'grid_mapping', 'LayerDepths', 'xc', 'yc', 'zc', 'depth', 'DXxDYy', 'u_velocity', 'v_velocity', 'elevation', 'temperature', 'salinity', 'ice_thickness', 'ice_compactness', 'salinity_ice']


In [3]:
#Create bottom-features 

def process_bottom_layer(filepath,
    variable_name,
    gridLons=None,  # To calculate statistical northness and eastness
    output_path=None
):
    """
    Process bottom layer data for a specified variable in a NetCDF file.

    Parameters:
    - filepath (str): Path to the NetCDF file.
    - variable_name (str): Name of the variable to process.
    - output_path (str): Path to save the processed file (optional). If None, the result is not saved.
    
    Returns:
    - xarray.DataArray: The time-averaged bottom layer data.
    """

    # Check if output path is valid
    if output_path is not None:
        output_dir = os.path.dirname(output_path)
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)
        elif not os.access(output_dir, os.W_OK):
            raise PermissionError(f"Write permission denied for the directory: {output_dir}")

    time_start = time.time()

    ds = xr.open_dataset(filepath)
    print(f"\nAccessed the dataset after {time.time() - time_start:.2f} seconds")

    # Drop NaN values along specified dimensions
    #for dim in ["time", "xc", "yc"]:
     #   if dim in ds.dims:  
      #      ds = ds.dropna(dim=dim, how="any")

    # Extract the desired variable
    if variable_name == "current_speed" or variable_name in ["statistical_northness", "statistical_eastness"]:
        data_var = ds["u_velocity"]
    elif variable_name == "temperature_sundahl":
        data_var = ds["temperature"]
    else:
        data_var = ds[variable_name]

    # Extract the first time step
    time_slice = data_var.isel(time=0)

    # Create a mask for valid values in the first time step
    valid_mask = ~time_slice.isnull()

    # Find the index of the bottom-most valid layer for each (yc, xc)
    bottom_layer_idx = valid_mask.argmin(dim="zc") - 1

    # Ensure bottom_layer_idx does not go negative
    bottom_layer_idx = bottom_layer_idx.clip(min=0)

    # Extract the bottom layer data across all time steps
    if variable_name == "current_speed":
        bottom_layer_data = (data_var.isel(zc=bottom_layer_idx) ** 2 + ds["v_velocity"].isel(zc=bottom_layer_idx) ** 2) ** 0.5
    elif variable_name in ["statistical_northness", "statistical_eastness"]:
        longitude_of_projection_origin = ds["grid_mapping"].attrs["longitude_of_projection_origin"]
        theta = gridLons - longitude_of_projection_origin
        eastward_velocity = data_var.isel(zc=bottom_layer_idx) * np.cos(np.deg2rad(theta)) - ds["v_velocity"].isel(zc=bottom_layer_idx) * np.sin(np.deg2rad(theta))
        northward_velocity = data_var.isel(zc=bottom_layer_idx) * np.sin(np.deg2rad(theta)) + ds["v_velocity"].isel(zc=bottom_layer_idx) * np.cos(np.deg2rad(theta))
        aspect = np.arctan2(eastward_velocity, northward_velocity)

        if variable_name == 'statistical_eastness':
            bottom_layer_data = np.sin(aspect)
        else:
            bottom_layer_data = np.cos(aspect)
    else:
        bottom_layer_data = data_var.isel(zc=bottom_layer_idx)

    ds.close()

    print(f"\nExtracted the bottom layer data after {time.time() - time_start:.2f} seconds.\n\nStarting computation of statistics...")

    if variable_name == 'temperature_sundahl':
        print("variable is sundahl temp")

        march_may_data = bottom_layer_data.isel(time=slice(59, 151))
        oct_dec_data = bottom_layer_data.isel(time=slice(273, 365))
        print("extracted months")
        """
        # Convert time dimension to datetime64 if it's not already
        if not np.issubdtype(bottom_layer_data['time'].dtype, np.datetime64):
            bottom_layer_data['time'] = xr.decode_cf(bottom_layer_data)['time']
        
        # Extract March-May and October-December using time accessor
        march_may_data = bottom_layer_data.where(bottom_layer_data['time'].dt.month.isin([3, 4, 5]), drop=True)
        oct_dec_data = bottom_layer_data.where(bottom_layer_data['time'].dt.month.isin([10, 11, 12]), drop=True)
        """
        # Compute seasonal means
        mean_march_may = march_may_data.mean(dim="time", skipna=True)
        mean_oct_dec = oct_dec_data.mean(dim="time", skipna=True)
        print("means extracted")
    
        print(f"\nComputed statistics after {time.time() - time_start:.2f} seconds")
    
        # Concatenate with explicit naming
        stats_array = xr.concat([mean_march_may, mean_oct_dec], dim="stat").rename(f"{variable_name}_features")
        stats_array = stats_array.assign_coords(stat=["min_(mean_march_may)", "max_(mean_oct_dec)"])
    else:
        # Calculate statistics across time
        time_avg_bottom_layer = bottom_layer_data.mean(dim="time", skipna=True)

        # Calculate both 10th and 90th percentiles
        time_percentiles = bottom_layer_data.quantile([0.1, 0.9], dim="time", skipna=True)

        print(f"\nComputed statistics after {time.time() - time_start:.2f} seconds")

        # Concatenate mean and percentiles into a DataArray
        stats_array = xr.concat(
            [time_avg_bottom_layer, time_percentiles.sel(quantile=0.1).drop_vars("quantile"), 
             time_percentiles.sel(quantile=0.9).drop_vars("quantile")],
            dim="stat"
        ).rename(f"{variable_name}_features")
        stats_array = stats_array.assign_coords(stat=["mean", "10th_percentile", "90th_percentile"])

    # Save to output file if specified
    if output_path:
        stats_array.to_netcdf(output_path, mode='w')

    return stats_array

In [ ]:
#Run for desired features

#Run on temperature data (sundahl definition)
#process_bottom_layer("/cluster/projects/itk-SINMOD/coral-mapping/midnor/PhysStates_2019.nc", "temperature_sundahl", output_path="/cluster/home/maikents/features_midnor_2019/bottom_features/temperature_sundahl_bottom_features.nc")

#Run on statistical northness
#process_bottom_layer("/cluster/projects/itk-SINMOD/coral-mapping/midnor/PhysStates_2019.nc", "statistical_northness", gridLons, output_path="/cluster/home/maikents/features_midnor_2019/bottom_features/statistical_northness_bottom_features.nc")

#Run on statistical eastness
#process_bottom_layer("/cluster/projects/itk-SINMOD/coral-mapping/midnor/PhysStates_2019.nc", "statistical_eastness", gridLons, output_path="/cluster/home/maikents/features_midnor_2019/bottom_features/statistical_eastness_bottom_features.nc")

#Run on salinity
#process_bottom_layer("/cluster/projects/itk-SINMOD/coral-mapping/midnor/PhysStates_2019.nc", "salinity", gridLons, output_path="/cluster/home/maikents/features_midnor_2019/bottom_features/salinity_bottom_features.nc")

#Run on current speed
#process_bottom_layer("/cluster/projects/itk-SINMOD/coral-mapping/midnor/PhysStates_2019.nc", "current_speed", output_path="/cluster/home/maikents/features_midnor_2019/bottom_features/current_speed_bottom_features.nc")


In [4]:
temperature = xr.open_dataset("/cluster/home/maikents/features_midnor_2019/bottom_features/temperature_sundahl_bottom_features.nc")
statistical_northness = xr.open_dataset("/cluster/home/maikents/features_midnor_2019/bottom_features/statistical_northness_bottom_features.nc")
statistical_eastness = xr.open_dataset("/cluster/home/maikents/features_midnor_2019/bottom_features/statistical_eastness_bottom_features.nc")
salinity = xr.open_dataset("/cluster/home/maikents/features_midnor_2019/bottom_features/salinity_bottom_features.nc")
current_speed = xr.open_dataset("/cluster/home/maikents/features_midnor_2019/bottom_features/current_speed_bottom_features.nc")


In [ ]:
#Align SINMOD and EMOD data into same format. Returns a big array of all the features including EMOD and SINMOD, 
#with the same grid spacing and EPRSG.

In [5]:
#1: Load and combine all bottom features 
SINMOD_features = xr.Dataset({
    'bottom_temperature_sundahl_features': temperature["temperature_sundahl_features"],
    'bottom_salinity_features': salinity["salinity_features"],
    'bottom_current_features': current_speed["current_speed_features"],
    'bottom_statistical_northness_features': statistical_northness["statistical_northness_features"],
    'bottom_statistical_eastness_features': statistical_eastness["statistical_eastness_features"],
})

temperature.close()
salinity.close()
current_speed.close()
statistical_northness.close()
statistical_eastness.close()

del temperature
del salinity
del current_speed
del statistical_northness
del statistical_eastness


SINMOD_features = SINMOD_features.reset_coords(drop=True)

print(SINMOD_features)

<xarray.Dataset> Size: 74MB
Dimensions:                                (xc: 950, yc: 555, stat: 5)
Coordinates:
  * xc                                     (xc) float32 4kB 800.0 ... 7.6e+05
  * yc                                     (yc) float32 2kB 800.0 ... 4.44e+05
  * stat                                   (stat) <U20 400B '10th_percentile'...
Data variables:
    bottom_temperature_sundahl_features    (stat, yc, xc) float32 11MB nan .....
    bottom_salinity_features               (stat, yc, xc) float32 11MB nan .....
    bottom_current_features                (stat, yc, xc) float32 11MB nan .....
    bottom_statistical_northness_features  (stat, yc, xc) float64 21MB nan .....
    bottom_statistical_eastness_features   (stat, yc, xc) float64 21MB nan .....


In [9]:
#2: Get null land points as verification
null_land_points = sinmod_2019['temperature'].isel(time=0, zc=0).isnull().sum().values

ocean_points = sinmod_2019['temperature'].isel(time=0, zc=0).notnull().sum().values

print(f"Null land points: {null_land_points}")
print(f"Ocean points: {ocean_points}")
print(f"Total points: {null_land_points + ocean_points}")

Null land points: 102334
Ocean points: 424916
Total points: 527250


In [6]:
#3: Get the SINMOD crs and attach it to the dataset

def obtain_sinmod_crs(PhysStates_data):
    grid_mapping = PhysStates_data['grid_mapping']  #Replace 'grid_mapping' with the correct variable name if different
    grid_attrs = grid_mapping.attrs  

    #Print horizontal resolution if available
    horizontal_resolution = grid_attrs.get('horizontal_resolution', 'unknown')
    print(f"\nHorizontal resolution: {horizontal_resolution} meters")

    #Construct the CRS using the attributes
    crs_sinmod = CRS.from_proj4(
        f"+proj=stere "
        f"+lat_0={grid_attrs['latitude_of_projection_origin']} "
        f"+lat_ts={grid_attrs['standard_parallel']} "
        f"+lon_0={grid_attrs['straight_vertical_longitude_from_pole']} "
        f"+x_0={grid_attrs['false_easting']} "
        f"+y_0={grid_attrs['false_northing']} "
        f"+a={grid_attrs['semi_major_axis']} "
        f"+b={grid_attrs['semi_minor_axis']} "
        f"+units=m +no_defs"
    )

    print(f"\nSINMOD CRS: {crs_sinmod}")
    return crs_sinmod

midnor_crs = obtain_sinmod_crs(sinmod_2019)

del(sinmod_2019)

#Attach the crs to the SINMOD dataset
SINMOD_features = SINMOD_features.rio.write_crs(midnor_crs)


Horizontal resolution: 800.0 meters

SINMOD CRS: +proj=stere +lat_0=90.0 +lat_ts=60.0 +lon_0=58.0 +x_0=2544800.0 +y_0=1918800.0 +a=6370000.0 +b=6370000.0 +units=m +no_defs +type=crs


In [7]:
#4: Align the SINMOD data with the bathymetry

tif_file = '/cluster/projects/itk-SINMOD/coral-mapping/data/raw_data/EMOD-tifs/bathymetry_32N_Clip_sample.tif'

tif_files = ['/cluster/projects/itk-SINMOD/coral-mapping/data/raw_data/EMOD-tifs/aspect_cos.tif', '/cluster/projects/itk-SINMOD/coral-mapping/data/raw_data/EMOD-tifs/aspect_sin.tif', '/cluster/projects/itk-SINMOD/coral-mapping/data/raw_data/EMOD-tifs/bathymetry_32N_Clip_sample.tif', '/cluster/projects/itk-SINMOD/coral-mapping/data/raw_data/EMOD-tifs/broad_BPI_std.tif',
             '/cluster/projects/itk-SINMOD/coral-mapping/data/raw_data/EMOD-tifs/fine_BPI_std.tif', '/cluster/projects/itk-SINMOD/coral-mapping/data/raw_data/EMOD-tifs/log_ruggedness_1.tif', '/cluster/projects/itk-SINMOD/coral-mapping/data/raw_data/EMOD-tifs/slope.tif']


def align_SINMOD_and_bathymetry(SINMOD_features, tif_file, resampling=Resampling.bilinear):
        
    with rioxarray.open_rasterio(tif_file) as tif:

        #Remove the band dimension from the tif data
        if 'band' in tif.dims:
            tif = tif.isel(band=0)

        SINMOD_features_reprojected = SINMOD_features.rio.reproject_match(tif, resampling=resampling)

        print(SINMOD_features_reprojected.rio.bounds())

        
        sinmod_mask = SINMOD_features_reprojected['bottom_temperature_sundahl_features'][0].isnull()

        #Apply the mask to the EMOD data
        tif = tif.where(~sinmod_mask, np.nan)

        valid_mask = SINMOD_features_reprojected['bottom_temperature_sundahl_features'][0].notnull()

        valid_columns = valid_mask.any(dim="y")  
        min_col = valid_columns.argmax().item()  #First non-NaN column from the left
        max_col = valid_columns.shape[0] - valid_columns[::-1].argmax().item() - 1  #First non-NaN column from the right

        #Find the first valid row (non-NaN) from the top (min row index)
        valid_rows = valid_mask.any(dim="x")  #Check for valid values in each row
        min_row = valid_rows.argmax().item()  #First non-NaN row from the top
        max_row = valid_rows.shape[0] - valid_rows[::-1].argmax().item() - 1  #Adjust for reverse indexing

        #Slice the raster to the bounding box of valid data
        clipped_SINMOD_features = SINMOD_features_reprojected.isel(x=slice(min_col, max_col + 1), y=slice(min_row, max_row + 1))
        
        clipped_tif = tif.rio.clip_box(minx=clipped_SINMOD_features.rio.bounds()[0]+1, 
                                        miny=clipped_SINMOD_features.rio.bounds()[1], 
                                        maxx=clipped_SINMOD_features.rio.bounds()[2], 
                                        maxy=clipped_SINMOD_features.rio.bounds()[3])
        
        clipped_tif = clipped_tif.reset_coords(drop=True)

    return clipped_SINMOD_features, clipped_tif

SINMOD_features_reprojected, tif = align_SINMOD_and_bathymetry(SINMOD_features, tif_file)

(-148857.58868623473, 6580002.847672726, 866884.4113137652, 7621679.847672726)


In [11]:
print(list(SINMOD_features_reprojected.variables))
print(type(SINMOD_features_reprojected))

['stat', 'spatial_ref', 'x', 'y', 'bottom_temperature_sundahl_features', 'bottom_salinity_features', 'bottom_current_features', 'bottom_statistical_northness_features', 'bottom_statistical_eastness_features']
<class 'xarray.core.dataset.Dataset'>


In [19]:
#5: Handle missing values in the SINMOD dataset, make sure NaNs are consistent throughout

#Check that all features have the same null points
assert (SINMOD_features_reprojected['bottom_temperature_sundahl_features'].isnull() == 
    SINMOD_features_reprojected['bottom_salinity_features'].isnull()).all()
assert (SINMOD_features_reprojected['bottom_temperature_sundahl_features'].isnull() == 
    SINMOD_features_reprojected['bottom_current_features'].isnull()).all()

#Check that if one is not null, then they are all not null
assert (SINMOD_features_reprojected['bottom_temperature_sundahl_features'].notnull() == 
    SINMOD_features_reprojected['bottom_salinity_features'].notnull()).all()
assert (SINMOD_features_reprojected['bottom_temperature_sundahl_features'].notnull() == 
    SINMOD_features_reprojected['bottom_current_features'].notnull()).all()

#Ensure that tif is NaN everywhere SINMOD_features_reprojected is NaN
assert np.all(np.isnan(tif.values) == np.isnan(SINMOD_features_reprojected['bottom_temperature_sundahl_features'][0].values))

#Ensure that tif is non-Nan everywhere SINMOD_features_reprojected is non-NaN
assert np.all(np.isfinite(tif.values) == np.isfinite(SINMOD_features_reprojected['bottom_temperature_sundahl_features'][0].values))

AssertionError: 

In [9]:
#6: Check tif properties
def check_tif_properties(tif_files):

    with rioxarray.open_rasterio(tif_files[0]) as tif:
        print("\n File to compare is: ", tif_files[0])
        ref_bounds, ref_crs, ref_res, ref_dims = tif.rio.bounds(), tif.rio.crs, tif.rio.resolution(), tif.shape

        #Check if all files match the reference properties
        for tif_file in tif_files[1:]:
            print("\n Checking file: ", tif_file)
            with rioxarray.open_rasterio(tif_file) as tif:
                if not (tif.rio.bounds() == ref_bounds and
                        tif.rio.crs == ref_crs and
                        tif.rio.resolution() == ref_res and
                        tif.shape == ref_dims):
                    print(f"Mismatch found in {tif_file}")
                    print(f"Expected bounds: {ref_bounds}, Found: {tif.rio.bounds()}")
                    print(f"Expected CRS: {ref_crs}, Found: {tif.rio.crs}")
                    print(f"Expected resolution: {ref_res}, Found: {tif.rio.resolution()}")
                    print(f"Expected dimensions: {ref_dims}, Found: {tif.shape}")
                    continue
                print("Properties match.")

check_tif_properties(tif_files)


 File to compare is:  /cluster/projects/itk-SINMOD/coral-mapping/data/raw_data/EMOD-tifs/aspect_cos.tif

 Checking file:  /cluster/projects/itk-SINMOD/coral-mapping/data/raw_data/EMOD-tifs/aspect_sin.tif
Properties match.

 Checking file:  /cluster/projects/itk-SINMOD/coral-mapping/data/raw_data/EMOD-tifs/bathymetry_32N_Clip_sample.tif
Mismatch found in /cluster/projects/itk-SINMOD/coral-mapping/data/raw_data/EMOD-tifs/bathymetry_32N_Clip_sample.tif
Expected bounds: (-65592.58868623473, 6656715.847672726, 784256.4113137652, 7539233.847672726), Found: (-148857.58868623473, 6580002.847672726, 866884.4113137652, 7621679.847672726)
Expected CRS: EPSG:32632, Found: EPSG:32632
Expected resolution: (91.0, -91.0), Found: (91.0, -91.0)
Expected dimensions: (1, 9698, 9339), Found: (1, 11447, 11162)

 Checking file:  /cluster/projects/itk-SINMOD/coral-mapping/data/raw_data/EMOD-tifs/broad_BPI_std.tif
Properties match.

 Checking file:  /cluster/projects/itk-SINMOD/coral-mapping/data/raw_data/EMO

In [ ]:
#7: Repeat for all the EMOD tifs and create a dataset for all the clipped tif files

EMOD_features = xr.Dataset()

for file in tif_files:
    SINMOD_temp, file_tif = align_SINMOD_and_bathymetry(SINMOD_features, file)
    
    #Add the clipped tif as a new variable in the dataset
    file_name = file.split("/")[-1].split(".")[0] + "_clipped"
    var_name = file.split("/")[-1].split(".")[0]
    EMOD_features[var_name] = file_tif

    file_tif.to_netcdf(f'/cluster/home/maikents/features_midnor_2019/emod_features/{file_name}.nc', mode='w')

    # Clear memory of file_tif and SINMOD_temp
    del file_tif
    del SINMOD_temp

EMOD_features = EMOD_features.reset_coords(drop=True)
EMOD_features.to_netcdf('/cluster/home/maikents/features_midnor_2019/emod_features/EMOD_features.nc', mode='w')

(-65592.58868623473, 6656715.847672726, 784256.4113137652, 7539233.847672726)
(-65592.58868623473, 6656715.847672726, 784256.4113137652, 7539233.847672726)
(-148857.58868623473, 6580002.847672726, 866884.4113137652, 7621679.847672726)
(-65592.58868623473, 6656715.847672726, 784256.4113137652, 7539233.847672726)


In [8]:
EMOD_features = xr.Dataset()

In [9]:
SINMOD1, file_tif1 = align_SINMOD_and_bathymetry(SINMOD_features, '/cluster/projects/itk-SINMOD/coral-mapping/data/raw_data/EMOD-tifs/aspect_cos.tif')
EMOD_features['aspect_cos'] = file_tif1


(-65592.58868623473, 6656715.847672726, 784256.4113137652, 7539233.847672726)


In [10]:
SINMOD2, file_tif2 = align_SINMOD_and_bathymetry(SINMOD_features, '/cluster/projects/itk-SINMOD/coral-mapping/data/raw_data/EMOD-tifs/aspect_sin.tif')
EMOD_features['aspect_sin'] = file_tif2

(-65592.58868623473, 6656715.847672726, 784256.4113137652, 7539233.847672726)


In [11]:
SINMOD3, file_tif3 = align_SINMOD_and_bathymetry(SINMOD_features, '/cluster/projects/itk-SINMOD/coral-mapping/data/raw_data/EMOD-tifs/bathymetry_32N_Clip_sample.tif')
EMOD_features['bathymetry_32N'] = file_tif3

(-148857.58868623473, 6580002.847672726, 866884.4113137652, 7621679.847672726)


In [12]:
SINMOD4, file_tif4 = align_SINMOD_and_bathymetry(SINMOD_features, '/cluster/projects/itk-SINMOD/coral-mapping/data/raw_data/EMOD-tifs/broad_BPI_std.tif')
EMOD_features['broad_BPI_std'] = file_tif4

(-65592.58868623473, 6656715.847672726, 784256.4113137652, 7539233.847672726)


In [ ]:
EMOD_features.to_netcdf('/cluster/home/maikents/features_midnor_2019/emod_features/emod.nc', mode='w')

In [14]:
EMOD_features = xr.open_dataset('/cluster/home/maikents/features_midnor_2019/emod_features/emod.nc')

In [15]:
SINMOD5, file_tif5 = align_SINMOD_and_bathymetry(SINMOD_features, '/cluster/projects/itk-SINMOD/coral-mapping/data/raw_data/EMOD-tifs/fine_BPI_std.tif')
EMOD_features['fine_BPI_std'] = file_tif5

(-65592.58868623473, 6656715.847672726, 784256.4113137652, 7539233.847672726)


In [16]:
SINMOD6, file_tif6 = align_SINMOD_and_bathymetry(SINMOD_features_reprojected, '/cluster/projects/itk-SINMOD/coral-mapping/data/raw_data/EMOD-tifs/log_ruggedness_1.tif')
EMOD_features['log_ruggedness'] = file_tif6

(-148857.58868623473, 6580002.847672726, 866884.4113137652, 7621679.847672726)


In [17]:
SINMOD7, file_tif7 = align_SINMOD_and_bathymetry(SINMOD_features_reprojected, '/cluster/projects/itk-SINMOD/coral-mapping/data/raw_data/EMOD-tifs/slope.tif')
EMOD_features['slope'] = file_tif7

(-65592.58868623473, 6656715.847672726, 784256.4113137652, 7539233.847672726)


In [19]:
EMOD_features.to_netcdf('/cluster/home/maikents/features_midnor_2019/emod_features/emod_complete.nc', mode='w')

In [5]:
EMOD_features = xr.open_dataset('/cluster/home/maikents/features_midnor_2019/emod_features/emod_complete.nc')

In [ ]:
#Creating the current - aspect angle feature. Absolute difference between the current direction and the depth aspect direction

aspect_bathymetry = EMOD_features['aspect_cos_clipped']
aspect_current = SINMOD_features_reprojected['bottom_statistical_northness_features'].sel(stat='mean')

SINMOD_features_reprojected['current_aspect_angle'] = abs(np.arccos(aspect_current) - np.arccos(aspect_bathymetry)) * 180 / np.pi

In [25]:
print(list(SINMOD_features_reprojected.variables))

['stat', 'x', 'y', 'spatial_ref', 'bottom_temperature_sundahl_features', 'bottom_salinity_features', 'bottom_current_features', 'bottom_statistical_northness_features', 'bottom_statistical_eastness_features', 'current_aspect_angle']


In [ ]:
#10: Saving the reprojected, unstandardized data
output_file = '/cluster/home/maikents/features_midnor_2019/ready_for_training/SINMOD_bottom_features_emod_grid.nc'
SINMOD_features_reprojected.to_netcdf(output_file, mode='w')

In [8]:
sinmod_unstandardized = xr.open_dataset('/cluster/home/maikents/features_midnor_2019/ready_for_training/SINMOD_bottom_features_emod_grid.nc')

In [3]:
print(list(sinmod_unstandardized.variables))

['stat', 'x', 'y', 'spatial_ref', 'bottom_temperature_sundahl_features', 'bottom_salinity_features', 'bottom_current_features', 'bottom_statistical_northness_features', 'bottom_statistical_eastness_features', 'current_aspect_angle']


In [ ]:
#Standardize temperature

temperature = sinmod_unstandardized.variables['bottom_temperature_sundahl_features'][:]

temp_data= temperature.astype(np.float32)  
temp_data_scaled = np.copy(temp_data)    

reshaped_temp_data = temp_data.values.reshape(temp_data.shape[0], -1)  

mask = np.isnan(reshaped_temp_data)

temp_scaler = StandardScaler()
for i in range(reshaped_temp_data.shape[1]):  
    column = reshaped_temp_data[:, i] 
    if not np.all(np.isnan(column)):  
        reshaped_temp_data[:, i][~mask[:, i]] = temp_scaler.fit_transform(column[~mask[:, i]].reshape(-1, 1)).ravel()


temp_data_scaled = reshaped_temp_data.reshape(temp_data.shape)

temperature_standardized = xr.DataArray(temp_data_scaled, dims=temperature.dims, attrs=temperature.attrs)

print(f"Standardised Temperature - Mean: {temperature_standardized.mean():.2f}, Std Dev: {temperature_standardized.std():.2f}")


In [16]:
temperature_standardized.to_netcdf('/cluster/home/maikents/features_midnor_2019/ready_for_training/standardized_single_features/standardized_sundahl_temperature.nc', mode='w')

In [3]:
#Standardize salinity

salinity = sinmod_unstandardized.variables['bottom_salinity_features'][:]

sal_data= salinity.astype(np.float32)  
sal_data_scaled = np.copy(sal_data)    

reshaped_sal_data = sal_data.values.reshape(sal_data.shape[0], -1)  

mask = np.isnan(reshaped_sal_data)

sal_scaler = StandardScaler()
for i in range(reshaped_sal_data.shape[1]):  
    column = reshaped_sal_data[:, i] 
    if not np.all(np.isnan(column)):  
        reshaped_sal_data[:, i][~mask[:, i]] = sal_scaler.fit_transform(column[~mask[:, i]].reshape(-1, 1)).ravel()


sal_data_scaled = reshaped_sal_data.reshape(sal_data.shape)

salinity_standardized = xr.DataArray(sal_data_scaled, dims=salinity.dims, attrs=salinity.attrs)

print(f"Standardised Salinity - Mean: {salinity_standardized.mean():.2f}, Std Dev: {salinity_standardized.std():.2f}")


Standardised Salinity - Mean: 0.00, Std Dev: 1.00


In [4]:
salinity_standardized.to_netcdf('/cluster/home/maikents/features_midnor_2019/ready_for_training/standardized_single_features/standardized_salinity.nc', mode='w')

In [5]:
#Standardize current speed

current = sinmod_unstandardized.variables['bottom_current_features'][:]

cur_data= current.astype(np.float32)  
cur_data_scaled = np.copy(cur_data)    

reshaped_cur_data = cur_data.values.reshape(cur_data.shape[0], -1)  

mask = np.isnan(reshaped_cur_data)

cur_scaler = StandardScaler()
for i in range(reshaped_cur_data.shape[1]):  
    column = reshaped_cur_data[:, i] 
    if not np.all(np.isnan(column)):  
        reshaped_cur_data[:, i][~mask[:, i]] = cur_scaler.fit_transform(column[~mask[:, i]].reshape(-1, 1)).ravel()


cur_data_scaled = reshaped_cur_data.reshape(cur_data.shape)

current_standardized = xr.DataArray(cur_data_scaled, dims=current.dims, attrs=current.attrs)

print(f"Standardised Current Speed - Mean: {current_standardized.mean():.2f}, Std Dev: {current_standardized.std():.2f}")


Standardised Current Speed - Mean: -0.00, Std Dev: 1.00


In [6]:
current_standardized.to_netcdf('/cluster/home/maikents/features_midnor_2019/ready_for_training/standardized_single_features/standardized_current_speed.nc', mode='w')

In [3]:
#Standardize statistical northness

north = sinmod_unstandardized.variables['bottom_statistical_northness_features'][:]

north_data= north.astype(np.float32)  
north_data_scaled = np.copy(north_data)    

reshaped_north_data = north_data.values.reshape(north_data.shape[0], -1)  

mask = np.isnan(reshaped_north_data)

north_scaler = StandardScaler()
for i in range(reshaped_north_data.shape[1]):  
    column = reshaped_north_data[:, i] 
    if not np.all(np.isnan(column)):  
        reshaped_north_data[:, i][~mask[:, i]] = north_scaler.fit_transform(column[~mask[:, i]].reshape(-1, 1)).ravel()


north_data_scaled = reshaped_north_data.reshape(north_data.shape)

north_standardized = xr.DataArray(north_data_scaled, dims=north.dims, attrs=north.attrs)

print(f"Standardised Statistical Northness - Mean: {north_standardized.mean():.2f}, Std Dev: {north_standardized.std():.2f}")


Standardised Statistical Northness - Mean: -0.00, Std Dev: 1.00


In [4]:
north_standardized.to_netcdf('/cluster/home/maikents/features_midnor_2019/ready_for_training/standardized_single_features/standardized_statistical_northness.nc', mode='w')

In [ ]:
#Standardize statistical eastness

east = sinmod_unstandardized.variables['bottom_statistical_eastness_features'][:]

east_data= east.astype(np.float32)  
east_data_scaled = np.copy(east_data)    

reshaped_east_data = east_data.values.reshape(east_data.shape[0], -1)  

mask = np.isnan(reshaped_east_data)

east_scaler = StandardScaler()
for i in range(reshaped_east_data.shape[1]):  
    column = reshaped_east_data[:, i] 
    if not np.all(np.isnan(column)):  
        reshaped_east_data[:, i][~mask[:, i]] = east_scaler.fit_transform(column[~mask[:, i]].reshape(-1, 1)).ravel()


east_data_scaled = reshaped_east_data.reshape(east_data.shape)

east_standardized = xr.DataArray(east_data_scaled, dims=east.dims, attrs=east.attrs)

print(f"Standardised Statistical Eastness - Mean: {east_standardized.mean():.2f}, Std Dev: {east_standardized.std():.2f}")


In [ ]:
east_standardized.to_netcdf('/cluster/home/maikents/features_midnor_2019/ready_for_training/standardized_single_features/standardized_statistical_eastness.nc', mode='w')

In [14]:
#Get current aspect angle
sinmod_bottom = xr.open_dataset('/cluster/projects/itk-SINMOD/coral-mapping/data/processed_data/features/ready-for-training/SINMOD_bottom_features.nc')

In [15]:
current_aspect_angle = sinmod_bottom.variables['current_aspect_angle']

In [23]:
#Standardize current aspect angle

ang = current_aspect_angle[:]

ang_data= ang.astype(np.float32)  
ang_data_scaled = np.copy(ang_data)    

reshaped_ang_data = ang_data.values.reshape(ang_data.shape[0], -1)  

mask = np.isnan(reshaped_ang_data)

ang_scaler = StandardScaler()
for i in range(reshaped_ang_data.shape[1]):  
    column = reshaped_ang_data[:, i] 
    if not np.all(np.isnan(column)):  
        reshaped_ang_data[:, i][~mask[:, i]] = ang_scaler.fit_transform(column[~mask[:, i]].reshape(-1, 1)).ravel()


ang_data_scaled = reshaped_ang_data.reshape(ang_data.shape)

ang_standardized = xr.DataArray(ang_data_scaled, dims=ang.dims, attrs=ang.attrs)

print(f"Standardised Current Aspect Angle - Mean: {ang_standardized.mean():.2f}, Std Dev: {ang_standardized.std():.2f}")

Standardised Current Aspect Angle - Mean: 0.00, Std Dev: 1.00


In [24]:
ang_standardized.to_netcdf('/cluster/home/maikents/features_midnor_2019/ready_for_training/standardized_single_features/standardized_current_aspect_angle.nc', mode='w')

In [5]:
#Combine standardized features
sinmod_unstandardized = xr.open_dataset('/cluster/home/maikents/features_midnor_2019/ready_for_training/SINMOD_bottom_features_emod_grid.nc')
temperature = xr.open_dataset('/cluster/home/maikents/features_midnor_2019/ready_for_training/standardized_single_features/standardized_sundahl_temperature.nc')
salinity = xr.open_dataset('/cluster/home/maikents/features_midnor_2019/ready_for_training/standardized_single_features/standardized_salinity.nc')
current = xr.open_dataset('/cluster/home/maikents/features_midnor_2019/ready_for_training/standardized_single_features/standardized_current_speed.nc')
statistical_northness = xr.open_dataset('/cluster/home/maikents/features_midnor_2019/ready_for_training/standardized_single_features/standardized_statistical_northness.nc')
statistical_eastness = xr.open_dataset('/cluster/home/maikents/features_midnor_2019/ready_for_training/standardized_single_features/standardized_statistical_eastness.nc')
current_aspect_angle = xr.open_dataset('/cluster/home/maikents/features_midnor_2019/ready_for_training/standardized_single_features/standardized_current_aspect_angle.nc')

In [22]:
sinmod_standardized = sinmod_unstandardized.copy()

sinmod_standardized['bottom_temperature_sundahl_features'] = temperature['__xarray_dataarray_variable__']
sinmod_standardized['bottom_salinity_features'] = salinity['__xarray_dataarray_variable__']
sinmod_standardized['bottom_current_features'] = current['__xarray_dataarray_variable__']
sinmod_standardized['bottom_statistical_northness_features'] = statistical_northness['__xarray_dataarray_variable__']
sinmod_standardized['bottom_statistical_eastness_features'] = statistical_eastness['__xarray_dataarray_variable__']
sinmod_standardized = sinmod_standardized.drop_vars('current_aspect_angle') #current_aspect_angle has to be loaded separately later!

#saving it
sinmod_standardized.to_netcdf('/cluster/home/maikents/features_midnor_2019/ready_for_training/SINMOD_bottom_features_emod_grid_standardized.nc')

In [8]:
#Standardize aspect cos

aspect_cos = EMOD_features.variables['aspect_cos'][:]

aspect_cos_data= aspect_cos.astype(np.float32)  
aspect_cos_data_scaled = np.copy(aspect_cos_data)    

reshaped_aspect_cos_data = aspect_cos_data.values.reshape(aspect_cos_data.shape[0], -1)  

mask = np.isnan(reshaped_aspect_cos_data)

aspect_cos_scaler = StandardScaler()
for i in range(reshaped_aspect_cos_data.shape[1]):  
    column = reshaped_aspect_cos_data[:, i] 
    if not np.all(np.isnan(column)):  
        reshaped_aspect_cos_data[:, i][~mask[:, i]] = aspect_cos_scaler.fit_transform(column[~mask[:, i]].reshape(-1, 1)).ravel()


aspect_cos_data_scaled = reshaped_aspect_cos_data.reshape(aspect_cos_data.shape)

aspect_cos_standardized = xr.DataArray(aspect_cos_data_scaled, dims=aspect_cos.dims, attrs=aspect_cos.attrs)

print(f"Standardised Aspect Cos - Mean: {aspect_cos_standardized.mean():.2f}, Std Dev: {aspect_cos_standardized.std():.2f}")


Standardised Aspect Cos - Mean: nan, Std Dev: nan


/cluster/home/maikents/PyEnvCoralMapping/lib/python3.12/site-packages/numpy/lib/_nanfunctions_impl.py:2019: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


In [ ]:
east_standardized.to_netcdf('/cluster/home/maikents/features_midnor_2019/ready_for_training/standardized_single_features/standardized_statistical_eastness.nc', mode='w')

In [6]:
emod_data = xr.open_dataset("/cluster/projects/itk-SINMOD/coral-mapping/data/processed_data/features/EMOD_features.nc")


In [7]:
print(list(emod_data.variables))

['x', 'y', 'aspect_cos', 'aspect_sin', 'bathymetry', 'broad_BPI', 'fine_BPI', 'log_ruggedness', 'slope']


In [8]:
emod_unstandardized = emod_data.copy()

In [10]:
#Standardize aspect cos

aspect_cos = emod_unstandardized.variables['aspect_cos'][:]

aspect_cos_data= aspect_cos.astype(np.float32)  
aspect_cos_data_scaled = np.copy(aspect_cos_data)    

reshaped_aspect_cos_data = aspect_cos_data.values.reshape(aspect_cos_data.shape[0], -1)  

mask = np.isnan(reshaped_aspect_cos_data)

aspect_cos_scaler = StandardScaler()
for i in range(reshaped_aspect_cos_data.shape[1]):  
    column = reshaped_aspect_cos_data[:, i] 
    if not np.all(np.isnan(column)):  
        reshaped_aspect_cos_data[:, i][~mask[:, i]] = aspect_cos_scaler.fit_transform(column[~mask[:, i]].reshape(-1, 1)).ravel()


aspect_cos_data_scaled = reshaped_aspect_cos_data.reshape(aspect_cos_data.shape)

aspect_cos_standardized = xr.DataArray(aspect_cos_data_scaled, dims=aspect_cos.dims, attrs=aspect_cos.attrs)

print(f"Standardised Aspect Cos - Mean: {aspect_cos_standardized.mean():.2f}, Std Dev: {aspect_cos_standardized.std():.2f}")


Standardised Aspect Cos - Mean: -0.00, Std Dev: 1.00


In [11]:
aspect_cos_standardized.to_netcdf('/cluster/home/maikents/features_midnor_2019/ready_for_training/standardized_single_features/standardized_aspect_cos.nc', mode='w')

In [12]:
#Standardize aspect sin

aspect_sin = emod_unstandardized.variables['aspect_sin'][:]

aspect_sin_data= aspect_sin.astype(np.float32)  
aspect_sin_data_scaled = np.copy(aspect_sin_data)    

reshaped_aspect_sin_data = aspect_sin_data.values.reshape(aspect_sin_data.shape[0], -1)  

mask = np.isnan(reshaped_aspect_sin_data)

aspect_sin_scaler = StandardScaler()
for i in range(reshaped_aspect_sin_data.shape[1]):  
    column = reshaped_aspect_sin_data[:, i] 
    if not np.all(np.isnan(column)):  
        reshaped_aspect_sin_data[:, i][~mask[:, i]] = aspect_sin_scaler.fit_transform(column[~mask[:, i]].reshape(-1, 1)).ravel()


aspect_sin_data_scaled = reshaped_aspect_sin_data.reshape(aspect_sin_data.shape)

aspect_sin_standardized = xr.DataArray(aspect_sin_data_scaled, dims=aspect_sin.dims, attrs=aspect_sin.attrs)

print(f"Standardised Aspect Sin - Mean: {aspect_sin_standardized.mean():.2f}, Std Dev: {aspect_sin_standardized.std():.2f}")


Standardised Aspect Sin - Mean: 0.00, Std Dev: 1.00


In [13]:
aspect_sin_standardized.to_netcdf('/cluster/home/maikents/features_midnor_2019/ready_for_training/standardized_single_features/standardized_aspect_sin.nc', mode='w')

In [16]:
#Standardize bathymetry

bathymetry = emod_unstandardized.variables['bathymetry'][:]

bathymetry_data= bathymetry.astype(np.float32)  
bathymetry_data_scaled = np.copy(bathymetry_data)    

reshaped_bathymetry_data = bathymetry_data.values.reshape(bathymetry_data.shape[0], -1)  

mask = np.isnan(reshaped_bathymetry_data)

bathymetry_scaler = StandardScaler()
for i in range(reshaped_bathymetry_data.shape[1]):  
    column = reshaped_bathymetry_data[:, i] 
    if not np.all(np.isnan(column)):  
        reshaped_bathymetry_data[:, i][~mask[:, i]] = bathymetry_scaler.fit_transform(column[~mask[:, i]].reshape(-1, 1)).ravel()


bathymetry_data_scaled = reshaped_bathymetry_data.reshape(bathymetry_data.shape)

bathymetry_standardized = xr.DataArray(bathymetry_data_scaled, dims=bathymetry.dims, attrs=bathymetry.attrs)

print(f"Standardised Bathymetry - Mean: {bathymetry_standardized.mean():.2f}, Std Dev: {bathymetry_standardized.std():.2f}")


Standardised Bathymetry - Mean: -0.00, Std Dev: 1.00


In [17]:
bathymetry_standardized.to_netcdf('/cluster/home/maikents/features_midnor_2019/ready_for_training/standardized_single_features/standardized_bathymetry.nc', mode='w')

In [19]:
#Standardize Broad BPI

broad_BPI = emod_unstandardized.variables['broad_BPI'][:]

broad_BPI_data= broad_BPI.astype(np.float32)  
broad_BPI_data_scaled = np.copy(broad_BPI_data)    

reshaped_broad_BPI_data = broad_BPI_data.values.reshape(broad_BPI_data.shape[0], -1)  

mask = np.isnan(reshaped_broad_BPI_data)

broad_BPI_scaler = StandardScaler()
for i in range(reshaped_broad_BPI_data.shape[1]):  
    column = reshaped_broad_BPI_data[:, i] 
    if not np.all(np.isnan(column)):  
        reshaped_broad_BPI_data[:, i][~mask[:, i]] = broad_BPI_scaler.fit_transform(column[~mask[:, i]].reshape(-1, 1)).ravel()


broad_BPI_data_scaled = reshaped_broad_BPI_data.reshape(broad_BPI_data.shape)

broad_BPI_standardized = xr.DataArray(broad_BPI_data_scaled, dims=broad_BPI.dims, attrs=broad_BPI.attrs)

print(f"Standardised Broad BPI - Mean: {broad_BPI_standardized.mean():.2f}, Std Dev: {broad_BPI_standardized.std():.2f}")


Standardised Broad BPI - Mean: 0.00, Std Dev: 1.00


In [20]:
broad_BPI_standardized.to_netcdf('/cluster/home/maikents/features_midnor_2019/ready_for_training/standardized_single_features/standardized_broad_BPI.nc', mode='w')

In [21]:
#Standardize Fine BPI

fine_BPI = emod_unstandardized.variables['fine_BPI'][:]

fine_BPI_data= fine_BPI.astype(np.float32)  
fine_BPI_data_scaled = np.copy(fine_BPI_data)    

reshaped_fine_BPI_data = fine_BPI_data.values.reshape(fine_BPI_data.shape[0], -1)  

mask = np.isnan(reshaped_fine_BPI_data)

fine_BPI_scaler = StandardScaler()
for i in range(reshaped_fine_BPI_data.shape[1]):  
    column = reshaped_fine_BPI_data[:, i] 
    if not np.all(np.isnan(column)):  
        reshaped_fine_BPI_data[:, i][~mask[:, i]] = fine_BPI_scaler.fit_transform(column[~mask[:, i]].reshape(-1, 1)).ravel()


fine_BPI_data_scaled = reshaped_fine_BPI_data.reshape(fine_BPI_data.shape)

fine_BPI_standardized = xr.DataArray(fine_BPI_data_scaled, dims=fine_BPI.dims, attrs=fine_BPI.attrs)

print(f"Standardised Fine BPI - Mean: {fine_BPI_standardized.mean():.2f}, Std Dev: {fine_BPI_standardized.std():.2f}")


Standardised Fine BPI - Mean: -0.00, Std Dev: 1.00


In [22]:
fine_BPI_standardized.to_netcdf('/cluster/home/maikents/features_midnor_2019/ready_for_training/standardized_single_features/standardized_fine_BPI.nc', mode='w')

In [23]:
#Standardize Log Ruggedness

log_ruggedness = emod_unstandardized.variables['log_ruggedness'][:]

log_ruggedness_data= log_ruggedness.astype(np.float32)  
log_ruggedness_data_scaled = np.copy(log_ruggedness_data)    

reshaped_log_ruggedness_data = log_ruggedness_data.values.reshape(log_ruggedness_data.shape[0], -1)  

mask = np.isnan(reshaped_log_ruggedness_data)

log_ruggedness_scaler = StandardScaler()
for i in range(reshaped_log_ruggedness_data.shape[1]):  
    column = reshaped_log_ruggedness_data[:, i] 
    if not np.all(np.isnan(column)):  
        reshaped_log_ruggedness_data[:, i][~mask[:, i]] = log_ruggedness_scaler.fit_transform(column[~mask[:, i]].reshape(-1, 1)).ravel()


log_ruggedness_data_scaled = reshaped_log_ruggedness_data.reshape(log_ruggedness_data.shape)

log_ruggedness_standardized = xr.DataArray(log_ruggedness_data_scaled, dims=log_ruggedness.dims, attrs=log_ruggedness.attrs)

print(f"Standardised Log Ruggedness - Mean: {log_ruggedness_standardized.mean():.2f}, Std Dev: {log_ruggedness_standardized.std():.2f}")


Standardised Log Ruggedness - Mean: -0.00, Std Dev: 1.00


In [24]:
log_ruggedness_standardized.to_netcdf('/cluster/home/maikents/features_midnor_2019/ready_for_training/standardized_single_features/standardized_log_ruggedness.nc', mode='w')

In [25]:
#Standardize slope

slope = emod_unstandardized.variables['slope'][:]

slope_data= slope.astype(np.float32)  
slope_data_scaled = np.copy(slope_data)    

reshaped_slope_data = slope_data.values.reshape(slope_data.shape[0], -1)  

mask = np.isnan(reshaped_slope_data)

slope_scaler = StandardScaler()
for i in range(reshaped_slope_data.shape[1]):  
    column = reshaped_slope_data[:, i] 
    if not np.all(np.isnan(column)):  
        reshaped_slope_data[:, i][~mask[:, i]] = slope_scaler.fit_transform(column[~mask[:, i]].reshape(-1, 1)).ravel()


slope_data_scaled = reshaped_slope_data.reshape(slope_data.shape)

slope_standardized = xr.DataArray(slope_data_scaled, dims=slope.dims, attrs=slope.attrs)

print(f"Standardised Slope - Mean: {slope_standardized.mean():.2f}, Std Dev: {slope_standardized.std():.2f}")


Standardised Slope - Mean: 0.00, Std Dev: 1.00


In [26]:
slope_standardized.to_netcdf('/cluster/home/maikents/features_midnor_2019/ready_for_training/standardized_single_features/standardized_slope.nc', mode='w')

In [28]:
aspect_cos_standardized = xr.open_dataset('/cluster/home/maikents/features_midnor_2019/ready_for_training/standardized_single_features/standardized_aspect_cos.nc')
aspect_sin_standardized = xr.open_dataset('/cluster/home/maikents/features_midnor_2019/ready_for_training/standardized_single_features/standardized_aspect_sin.nc')
bathymetry_standardized = xr.open_dataset('/cluster/home/maikents/features_midnor_2019/ready_for_training/standardized_single_features/standardized_bathymetry.nc')
broad_BPI_standardized = xr.open_dataset('/cluster/home/maikents/features_midnor_2019/ready_for_training/standardized_single_features/standardized_broad_BPI.nc')
fine_BPI_standardized = xr.open_dataset('/cluster/home/maikents/features_midnor_2019/ready_for_training/standardized_single_features/standardized_fine_BPI.nc')
log_ruggedness_standardized = xr.open_dataset('/cluster/home/maikents/features_midnor_2019/ready_for_training/standardized_single_features/standardized_log_ruggedness.nc')
slope_standardized = xr.open_dataset('/cluster/home/maikents/features_midnor_2019/ready_for_training/standardized_single_features/standardized_slope.nc')


In [30]:
emod_standardized = emod_unstandardized.copy()

emod_standardized['aspect_cos'] = aspect_cos_standardized['__xarray_dataarray_variable__']
emod_standardized['aspect_sin'] = aspect_sin_standardized['__xarray_dataarray_variable__']
emod_standardized['bathymetry'] = bathymetry_standardized['__xarray_dataarray_variable__']
emod_standardized['broad_BPI'] = broad_BPI_standardized['__xarray_dataarray_variable__']
emod_standardized['fine_BPI'] = fine_BPI_standardized['__xarray_dataarray_variable__']
emod_standardized['log_ruggedness'] = log_ruggedness_standardized['__xarray_dataarray_variable__']
emod_standardized['slope'] = slope_standardized['__xarray_dataarray_variable__']

#saving it
emod_standardized.to_netcdf('/cluster/home/maikents/features_midnor_2019/ready_for_training/EMOD_standardized.nc')